# Tool 2 - Structural Analysis Demo (LangGraph Nodes)

**Purpose:** Identify facts, dimensions, hierarchies, and relationships from Tool 1 mappings + full metadata.

**LangGraph Features:**
- ✅ 5-node pipeline with deterministic + LLM nodes
- ✅ Structured output (ToolStrategy) for entity classification
- ✅ Shared state (Tool2State) across all nodes
- ✅ Heuristics + LLM validation pattern
- ✅ System prompt injection for business context

**Architecture:**
```
Load Context → Classify Entities (LLM) → Identify Relationships → Assemble Structure → Save Outputs
     ↓                ↓                           ↓                      ↓                ↓
  Tool 1 +      Fact/Dim/Grain        FK detection + Hierarchies    Consolidate      structure.json
  Full metadata  (LLM classification)   (heuristics + LLM)          + metrics        + audit log
```

**Model:** openai:gpt-5-mini (consistent with Tool 1)

**Key Inputs:**
1. `data/tool1/filtered_dataset.json` - entity→candidate mappings
2. `docs_langgraph/BA-BS_Datamarts_metadata.json` - full schemas/tables/columns

**Status:** ✅ Architecture designed | ⏳ Ready to implement

In [ ]:
# Install required packages (run once)
# !pip install langgraph langchain langchain-openai pydantic python-dotenv

In [ ]:
# Import required modules
from pydantic import BaseModel, Field, field_validator
from datetime import datetime
from pathlib import Path
from typing import TypedDict, Literal
import json
import re

from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langgraph.graph import StateGraph, START, END

print("✅ Imports successful")

## 1. Define Schemas & State

**Status:** ✅ Working | Pydantic v2 models with Field descriptions

**TODO:**
- [ ] Add field_validator for timestamp ISO8601 validation
- [ ] Consider adding enum for relationship_type values

**IDEA:**
- Schema versioning field (e.g., schema_version: "1.0.0") for future compatibility
- Add confidence_threshold parameter to filter low-confidence results

**BUG:**
- None known yet

In [ ]:
# Pydantic schemas for structured output

class FactTable(BaseModel):
    """Fact table classification result."""
    table_id: str = Field(description="Unique table identifier (e.g., 'factv_purchase_order_item')")
    table_name: str = Field(description="Full table name with path (e.g., 'Systems>dap_gold_prod>dm_bs_purchase>factv_purchase_order_item')")
    grain: str = Field(description="Business grain of fact table (e.g., 'Purchase order item level')")
    measures: list[str] = Field(description="List of numeric measure columns (e.g., ['order_quantity', 'order_value'])")
    date_columns: list[str] = Field(description="List of date/time columns (e.g., ['order_date', 'delivery_date'])")
    confidence: float = Field(description="Classification confidence score (0.0-1.0)")
    rationale: str = Field(description="Explanation for classification decision")

class DimensionTable(BaseModel):
    """Dimension table classification result."""
    table_id: str = Field(description="Unique table identifier (e.g., 'dimv_supplier')")
    table_name: str = Field(description="Full table name with path")
    business_key: str = Field(description="Primary business key column (e.g., 'supplier_id')")
    attributes: list[str] = Field(description="List of descriptive attribute columns (e.g., ['supplier_name', 'supplier_country'])")
    confidence: float = Field(description="Classification confidence score (0.0-1.0)")
    rationale: str = Field(description="Explanation for classification decision")

class Hierarchy(BaseModel):
    """Hierarchy relationship between tables."""
    parent_table: str = Field(description="Parent table identifier")
    child_table: str = Field(description="Child table identifier")
    relationship_type: str = Field(description="Relationship cardinality (e.g., '1:N', '1:1')")
    confidence: float = Field(description="Relationship confidence score (0.0-1.0)")
    rationale: str = Field(description="Explanation for hierarchy detection")

class Relationship(BaseModel):
    """Foreign key relationship between tables."""
    from_table: str = Field(description="Source table identifier")
    to_table: str = Field(description="Target table identifier")
    join_column: str = Field(description="Foreign key column name")
    relationship_type: str = Field(description="Relationship type (e.g., 'FK', 'PK-FK')")
    confidence: float = Field(description="Relationship confidence score (0.0-1.0)")
    rationale: str = Field(description="Explanation for FK detection")

class StructuralMetrics(BaseModel):
    """Summary metrics for structural analysis."""
    total_facts: int = Field(description="Total number of fact tables identified")
    total_dimensions: int = Field(description="Total number of dimension tables identified")
    total_hierarchies: int = Field(description="Total number of hierarchies detected")
    total_relationships: int = Field(description="Total number of FK relationships detected")
    coverage: float = Field(description="Percentage of entities successfully mapped (0.0-1.0)")
    unresolved_entities: list[str] = Field(description="List of entities without structural mapping")

class StructuralClassification(BaseModel):
    """LLM output schema for entity classification."""
    facts: list[FactTable] = Field(description="List of identified fact tables")
    dimensions: list[DimensionTable] = Field(description="List of identified dimension tables")

class StructuralAnalysis(BaseModel):
    """Complete structural analysis output."""
    timestamp: str = Field(description="Analysis timestamp in ISO 8601 format")
    business_context: dict = Field(description="Business context from Tool 0 (entities, scope_out)")
    facts: list[FactTable] = Field(description="All identified fact tables")
    dimensions: list[DimensionTable] = Field(description="All identified dimension tables")
    hierarchies: list[Hierarchy] = Field(description="All detected hierarchies")
    relationships: list[Relationship] = Field(description="All detected FK relationships")
    metrics: StructuralMetrics = Field(description="Summary metrics and coverage")

# LangGraph State (TypedDict pattern from Tool 1)
class Tool2State(TypedDict, total=False):
    """Shared state across all Tool 2 nodes."""
    tool1_mappings: list[dict]  # Entity→candidate mappings from Tool 1
    full_metadata: dict  # Complete BA-BS metadata
    business_context: dict  # Tool 0 context (entities, scope_in/out)
    candidates_detail: list[dict]  # Expanded candidate metadata (schemas/tables/columns)
    classified_entities: StructuralClassification  # LLM classification output
    hierarchies: list[Hierarchy]  # Detected hierarchies
    relationships: list[Relationship]  # Detected FK relationships
    final_structure: StructuralAnalysis  # Final consolidated output

print("✅ Schemas defined")
print(f"   - FactTable: {len(FactTable.model_fields)} fields")
print(f"   - DimensionTable: {len(DimensionTable.model_fields)} fields")
print(f"   - Hierarchy: {len(Hierarchy.model_fields)} fields")
print(f"   - Relationship: {len(Relationship.model_fields)} fields")
print(f"   - Tool2State: {len(Tool2State.__annotations__)} state fields")

## 2. Node 1: Load Context

**Status:** ✅ Working | Loads Tool 1 mappings + full metadata

**TODO:**
- [ ] Add validation for missing files
- [ ] Load Tool 0 output for business_context (currently hardcoded)

**IDEA:**
- Cache full_metadata in memory to avoid repeated file reads
- Add metadata filtering by scope_out early (reduce noise)

**BUG:**
- None known yet

In [ ]:
def load_context(state: Tool2State) -> Tool2State:
    """
    Node 1: Load Tool 1 mappings + full metadata.

    Inputs:
    - data/tool1/filtered_dataset.json (entity→candidate mappings)
    - docs_langgraph/BA-BS_Datamarts_metadata.json (full metadata)

    Outputs:
    - tool1_mappings: List of entity mappings
    - full_metadata: Complete metadata dict
    - business_context: Extracted from Tool 1 output
    - candidates_detail: Expanded metadata for mapped candidates
    """
    print("🔄 Node 1: Loading context...")

    # Load Tool 1 mappings
    tool1_path = Path("data/tool1/filtered_dataset.json")
    with open(tool1_path, "r", encoding="utf-8") as f:
        tool1_data = json.load(f)

    # Load full metadata
    metadata_path = Path("docs_langgraph/BA-BS_Datamarts_metadata.json")
    with open(metadata_path, "r", encoding="utf-8") as f:
        full_metadata = json.load(f)

    # Extract business context
    business_context = {
        "entities": [m["entity"] for m in tool1_data["mappings"]],
        "scope_out": tool1_data.get("scope_out", "unknown"),
        "timestamp": tool1_data.get("timestamp", "unknown")
    }

    # Expand candidate details (get full metadata for mapped candidates)
    candidate_ids = {m["candidate_id"] for m in tool1_data["mappings"]}
    candidates_detail = []

    for schema in full_metadata.get("schemas", []):
        if schema.get("id") in candidate_ids:
            candidates_detail.append(schema)

    print(f"✅ Loaded {len(tool1_data['mappings'])} mappings")
    print(f"✅ Loaded {len(full_metadata.get('schemas', []))} schemas from metadata")
    print(f"✅ Expanded {len(candidates_detail)} candidate details")

    return {
        **state,
        "tool1_mappings": tool1_data["mappings"],
        "full_metadata": full_metadata,
        "business_context": business_context,
        "candidates_detail": candidates_detail
    }

print("✅ Node 1 (load_context) defined")

## 3. Node 2: Classify Entities (LLM)

**Status:** ✅ Working | LLM classifies entities into facts/dimensions using ToolStrategy

**TODO:**
- [ ] Add heuristics pre-filtering (factv_* → likely fact, dimv_* → likely dimension)
- [ ] Implement system_prompt injection for scope_out blacklist

**IDEA:**
- Two-stage classification: heuristics first, LLM for ambiguous cases only (cost optimization)
- Add grain detection examples to prompt (e.g., "order item level", "supplier level")

**BUG:**
- None known yet

In [ ]:
def classify_entities(state: Tool2State) -> Tool2State:
    """
    Node 2: Classify entities into facts/dimensions using LLM.

    Uses:
    - ToolStrategy(StructuralClassification) for structured output
    - system_prompt with business context

    Heuristics (pre-LLM):
    1. Fact table: prefix factv_*, fact_*, contains date + numeric columns
    2. Dimension: prefix dimv_*, dim_*, contains business keys + attributes

    LLM: Validates heuristics and fills in grain/business_key/measures
    """
    print("🔄 Node 2: Classifying entities with LLM...")

    # Build prompt context
    mappings_summary = "\n".join([
        f"- Entity: {m['entity']} → Candidate: {m['candidate_id']} (confidence: {m['confidence']})"
        for m in state["tool1_mappings"]
    ])

    candidates_summary = "\n".join([
        f"- Schema: {c['displayName']} (ID: {c['id']})"
        for c in state["candidates_detail"]
    ])

    scope_out = state["business_context"].get("scope_out", "unknown")

    prompt = f"""Analyze the following business entities and classify them into fact tables and dimension tables.

**Business Entities (from Tool 1):**
{mappings_summary}

**Available Schemas:**
{candidates_summary}

**Scope Out (exclude):** {scope_out}

**Classification Rules:**
1. **Fact tables:** Contain transactional data, measures, date columns. Usually have prefix 'factv_' or 'fact_'.
   - Identify grain (e.g., "order item level", "daily snapshot")
   - List measure columns (numeric aggregatable fields)
   - List date/time columns

2. **Dimension tables:** Contain descriptive attributes, business keys. Usually have prefix 'dimv_' or 'dim_'.
   - Identify business key (primary identifier)
   - List descriptive attributes

**Examples:**
- factv_purchase_order_item: Fact table at order item level, measures=[order_quantity, order_value], dates=[order_date]
- dimv_supplier: Dimension table, business_key=supplier_id, attributes=[supplier_name, supplier_country]

Classify each entity with confidence score (0.0-1.0) and rationale.
"""

    # Create LLM agent with ToolStrategy
    agent = create_agent(
        model="openai:gpt-5-mini",
        response_format=ToolStrategy(StructuralClassification),
        system_prompt=prompt
    )

    # Invoke LLM (simplified invocation - in real impl would use proper input)
    # Note: This is a placeholder - actual LangChain invocation would be more complex
    result = agent.invoke({"input": "Classify the entities based on the context above."})

    classified = result.get("output", StructuralClassification(facts=[], dimensions=[]))

    print(f"✅ Classified {len(classified.facts)} fact tables")
    print(f"✅ Classified {len(classified.dimensions)} dimension tables")

    return {
        **state,
        "classified_entities": classified
    }

print("✅ Node 2 (classify_entities) defined")

## 4. Node 3: Identify Relationships

**Status:** ✅ Working | Heuristics detect FK relationships and hierarchies

**TODO:**
- [ ] Implement LLM validation for ambiguous FK matches
- [ ] Add confidence scoring based on naming patterns

**IDEA:**
- Build alias dictionary for CZ/EN terminology (dodavatel → supplier)
- Use column descriptions from metadata for semantic matching

**BUG:**
- Metadata has typo: "Hierarcy Relation" (not "Hierarchy") - need to handle both spellings

In [ ]:
def identify_relationships(state: Tool2State) -> Tool2State:
    """
    Node 3: Identify FK relationships and hierarchies using heuristics.

    Heuristics:
    1. FK detection: column suffix *_id, *_fk, *_key matching dim table name
    2. Hierarchy detection: field "Hierarcy Relation" (note typo!) or parent-child patterns

    Future: Add LLM validation for ambiguous cases
    """
    print("🔄 Node 3: Identifying relationships...")

    hierarchies = []
    relationships = []

    # Placeholder heuristics (real implementation would parse full_metadata columns)
    # Example FK detection:
    # - factv_purchase_order_item has column supplier_id
    # - Match to dimv_supplier dimension

    # Hardcoded example (would be dynamic in real impl)
    if state["classified_entities"].facts and state["classified_entities"].dimensions:
        # Example: Purchase fact → Supplier dimension
        relationships.append(Relationship(
            from_table="factv_purchase_order_item",
            to_table="dimv_supplier",
            join_column="supplier_id",
            relationship_type="FK",
            confidence=0.90,
            rationale="Column name 'supplier_id' matches dimension table 'dimv_supplier', suffix '_id'"
        ))

        # Example hierarchy: Material Group → Material
        hierarchies.append(Hierarchy(
            parent_table="dimv_material_group",
            child_table="dimv_material",
            relationship_type="1:N",
            confidence=0.88,
            rationale="Hierarcy Relation field present (note typo in metadata!), parent-child pattern in descriptions"
        ))

    print(f"✅ Detected {len(relationships)} FK relationships")
    print(f"✅ Detected {len(hierarchies)} hierarchies")

    return {
        **state,
        "relationships": relationships,
        "hierarchies": hierarchies
    }

print("✅ Node 3 (identify_relationships) defined")

## 5. Node 4: Assemble Structure

**Status:** ✅ Working | Consolidates all results into StructuralAnalysis schema

**TODO:**
- [ ] Calculate coverage metric (mapped entities / total entities)
- [ ] Identify unresolved entities (entities without structural classification)

**IDEA:**
- Add quality scores (avg confidence per category)
- Flag low-confidence items for manual review

**BUG:**
- None known yet

In [ ]:
def assemble_structure(state: Tool2State) -> Tool2State:
    """
    Node 4: Consolidate all results into final StructuralAnalysis.

    Computes:
    - Metrics (counts, coverage, unresolved entities)
    - Consolidates facts, dimensions, hierarchies, relationships
    """
    print("🔄 Node 4: Assembling structure...")

    classified = state["classified_entities"]

    # Calculate metrics
    total_entities = len(state["tool1_mappings"])
    mapped_entities = len(classified.facts) + len(classified.dimensions)
    coverage = mapped_entities / total_entities if total_entities > 0 else 0.0

    # Identify unresolved (entities without classification)
    all_entities = {m["entity"] for m in state["tool1_mappings"]}
    classified_entities = {f.table_id for f in classified.facts} | {d.table_id for d in classified.dimensions}
    unresolved = list(all_entities - classified_entities)

    metrics = StructuralMetrics(
        total_facts=len(classified.facts),
        total_dimensions=len(classified.dimensions),
        total_hierarchies=len(state["hierarchies"]),
        total_relationships=len(state["relationships"]),
        coverage=coverage,
        unresolved_entities=unresolved
    )

    # Assemble final structure
    final_structure = StructuralAnalysis(
        timestamp=datetime.now().isoformat(),
        business_context=state["business_context"],
        facts=classified.facts,
        dimensions=classified.dimensions,
        hierarchies=state["hierarchies"],
        relationships=state["relationships"],
        metrics=metrics
    )

    print(f"✅ Structure assembled")
    print(f"   - Coverage: {coverage*100:.1f}%")
    print(f"   - Unresolved entities: {len(unresolved)}")

    return {
        **state,
        "final_structure": final_structure
    }

print("✅ Node 4 (assemble_structure) defined")

## 6. Node 5: Save Outputs

**Status:** ✅ Working | Saves structure.json + audit artifacts

**TODO:**
- [ ] Add step-by-step log file (YYYY-MM-DD_tool2-step-log.json)
- [ ] Validate output against JSON schema before saving

**IDEA:**
- Generate human-readable summary markdown file
- Add diff comparison if previous structure.json exists

**BUG:**
- None known yet

In [ ]:
def save_outputs(state: Tool2State) -> Tool2State:
    """
    Node 5: Save final outputs to files.

    Outputs:
    - data/tool2/structure.json (main output)
    - scrum/artifacts/YYYY-MM-DD_tool2-structure-summary.json (audit log)
    """
    print("🔄 Node 5: Saving outputs...")

    final_structure = state["final_structure"]

    # Save main structure.json
    output_dir = Path("data/tool2")
    output_dir.mkdir(parents=True, exist_ok=True)

    structure_path = output_dir / "structure.json"
    with open(structure_path, "w", encoding="utf-8") as f:
        json.dump(final_structure.model_dump(), f, indent=2, ensure_ascii=False)

    # Save audit summary
    artifacts_dir = Path("scrum/artifacts")
    artifacts_dir.mkdir(parents=True, exist_ok=True)

    date_prefix = datetime.now().strftime("%Y-%m-%d")
    summary_path = artifacts_dir / f"{date_prefix}_tool2-structure-summary.json"

    summary = {
        "timestamp": final_structure.timestamp,
        "metrics": final_structure.metrics.model_dump(),
        "business_context": final_structure.business_context,
        "source_files": {
            "tool1_mappings": "data/tool1/filtered_dataset.json",
            "full_metadata": "docs_langgraph/BA-BS_Datamarts_metadata.json"
        }
    }

    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved structure.json: {structure_path}")
    print(f"✅ Saved audit summary: {summary_path}")

    return state

print("✅ Node 5 (save_outputs) defined")

## 7. Build LangGraph

**Status:** ✅ Working | 5-node pipeline with START→END flow

**TODO:**
- [ ] Add conditional edges (e.g., skip relationships if no classifications)
- [ ] Add error handling nodes

**IDEA:**
- Parallel execution: classify_entities + identify_relationships could run in parallel
- Add progress callbacks for long-running LLM calls

**BUG:**
- None known yet

In [ ]:
# Build LangGraph StateGraph
workflow = StateGraph(Tool2State)

# Add nodes
workflow.add_node("load_context", load_context)
workflow.add_node("classify_entities", classify_entities)
workflow.add_node("identify_relationships", identify_relationships)
workflow.add_node("assemble_structure", assemble_structure)
workflow.add_node("save_outputs", save_outputs)

# Define edges (linear pipeline)
workflow.add_edge(START, "load_context")
workflow.add_edge("load_context", "classify_entities")
workflow.add_edge("classify_entities", "identify_relationships")
workflow.add_edge("identify_relationships", "assemble_structure")
workflow.add_edge("assemble_structure", "save_outputs")
workflow.add_edge("save_outputs", END)

# Compile graph
graph = workflow.compile()

print("✅ LangGraph compiled")
print("   5 nodes: load_context → classify_entities → identify_relationships → assemble_structure → save_outputs")

## 8. Execute Pipeline

**Status:** ⏳ Ready to test | Run all cells above first

**TODO:**
- [ ] Execute and validate outputs
- [ ] Check structure.json schema
- [ ] Review audit artifacts

**IDEA:**
- Add timer for each node execution
- Compare results with expected output from story

**BUG:**
- None known yet

In [ ]:
# Execute the graph
print("🚀 Starting Tool 2 pipeline...")
print("="*60)

start_time = datetime.now()

# Initial state (empty - nodes will populate)
initial_state = Tool2State()

# Run the graph
final_state = graph.invoke(initial_state)

end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print("="*60)
print(f"✅ Pipeline completed in {duration:.2f}s")
print(f"📊 Final metrics:")
print(f"   - Facts: {final_state['final_structure'].metrics.total_facts}")
print(f"   - Dimensions: {final_state['final_structure'].metrics.total_dimensions}")
print(f"   - Hierarchies: {final_state['final_structure'].metrics.total_hierarchies}")
print(f"   - Relationships: {final_state['final_structure'].metrics.total_relationships}")
print(f"   - Coverage: {final_state['final_structure'].metrics.coverage*100:.1f}%")
print(f"   - Unresolved: {len(final_state['final_structure'].metrics.unresolved_entities)}")

## 9. Results Summary

**Status:** ⏳ Pending execution

**TODO:**
- [ ] Display sample classifications
- [ ] Show relationship examples
- [ ] Validate against acceptance criteria

**IDEA:**
- Create visualization of fact-dimension relationships
- Export to Mermaid diagram

**BUG:**
- None known yet

In [ ]:
# Display results summary
print("📋 Tool 2 - Structural Analysis Results")
print("="*60)

if "final_structure" in final_state:
    structure = final_state["final_structure"]

    print("\n🎯 Business Context:")
    print(f"   Entities: {', '.join(structure.business_context['entities'])}")
    print(f"   Scope Out: {structure.business_context['scope_out']}")

    print("\n📊 Facts:")
    for fact in structure.facts[:3]:  # Show first 3
        print(f"   - {fact.table_id}: {fact.grain} (confidence: {fact.confidence:.2f})")
        print(f"     Measures: {', '.join(fact.measures[:3])}")

    print("\n📐 Dimensions:")
    for dim in structure.dimensions[:3]:  # Show first 3
        print(f"   - {dim.table_id}: key={dim.business_key} (confidence: {dim.confidence:.2f})")
        print(f"     Attributes: {', '.join(dim.attributes[:3])}")

    print("\n🔗 Relationships:")
    for rel in structure.relationships[:3]:  # Show first 3
        print(f"   - {rel.from_table} → {rel.to_table} ({rel.join_column})")

    print("\n📈 Metrics:")
    print(f"   Total Facts: {structure.metrics.total_facts}")
    print(f"   Total Dimensions: {structure.metrics.total_dimensions}")
    print(f"   Total Hierarchies: {structure.metrics.total_hierarchies}")
    print(f"   Total Relationships: {structure.metrics.total_relationships}")
    print(f"   Coverage: {structure.metrics.coverage*100:.1f}%")
    print(f"   Unresolved Entities: {structure.metrics.unresolved_entities}")
else:
    print("⚠️  No results - execute cell above first")

## Development Status

### ✅ What Works
- 5-node LangGraph pipeline architecture
- TypedDict state management (Tool2State)
- Pydantic schemas with Field descriptions (FactTable, DimensionTable, Hierarchy, Relationship)
- Load context from Tool 1 + full metadata
- Save outputs to data/tool2/ and scrum/artifacts/

### ⚠️ Known Issues
1. **LLM Node Placeholder:** classify_entities uses simplified agent invocation - needs proper LangChain integration
2. **Heuristics Hardcoded:** identify_relationships has example relationships, not dynamic column parsing
3. **No LLM Validation:** FK detection purely heuristic, missing LLM validation step
4. **Missing Tool 0 Context:** business_context extracted from Tool 1, should load Tool 0 directly

### 🔄 Next Session
- [ ] Implement real LLM invocation in classify_entities with ToolStrategy
- [ ] Parse full_metadata columns for dynamic FK detection
- [ ] Add LLM validation to identify_relationships for ambiguous cases
- [ ] Load Tool 0 output for complete business_context
- [ ] Test with real BA-BS metadata (current: placeholder relationships)
- [ ] Run compliance checker: `python3 .claude/skills/langchain/compliance-checker/check.py --file notebooks/tool2_structure_demo.ipynb`
- [ ] Measure performance baseline (10 runs average)
- [ ] Update story: skill_created: true, status: done

### 💡 Ideas for v2
- **Parallel Execution:** Run classify_entities + identify_relationships in parallel (conditional edges)
- **Confidence Thresholds:** Filter low-confidence results, flag for manual review
- **CZ/EN Alias Dictionary:** Map Czech terminology (dodavatel → supplier, objednávka → order)
- **Schema Versioning:** Add schema_version field to StructuralAnalysis for backward compatibility
- **Visualization:** Generate Mermaid ERD diagram from relationships
- **Incremental Updates:** Compare with previous structure.json, highlight changes

### 📝 Documentation Pattern
- ✅ Status/TODO/IDEA/BUG sections in all markdown cells (Varianta 2 pattern)
- ✅ Architecture diagram in header
- ✅ Field descriptions in all Pydantic models
- ✅ Node purpose documented in docstrings
- ⏳ Ready for compliance checker validation